In [3]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Install ibm/tsfm
! pip install "granite-tsfm[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.22"

  Cloning https://github.com/ibm-granite/granite-tsfm.git (to revision v0.2.22) to /tmp/pip-install-l25hxxaa/granite-tsfm_e9b040ab003e4620985692f972fdddcf
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite/granite-tsfm.git /tmp/pip-install-l25hxxaa/granite-tsfm_e9b040ab003e4620985692f972fdddcf
  Running command git checkout -q 216850d0cb073e31689049c1334f701fe11bc2c3
  Resolved https://github.com/ibm-granite/granite-tsfm.git to commit 216850d0cb073e31689049c1334f701fe11bc2c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from TSB_AD.model_wrapper import run_Unsupervise_AD
from TSB_AD.evaluation.metrics import get_metrics

# Specify Anomaly Detector to use and data directory
AD_Name = 'TTM_ZS'   # It can be replaced with any anomaly detector available in TSB-AD
data_direc = 'Datasets/TSB-AD-M/115_PSM_id_1_Facility_tr_50000_1st_129872.csv'

# Loading DataIForest
df = pd.read_csv(data_direc).iloc[120001:140000].dropna() 
data = df.iloc[:, 0:-1].values.astype(float)
label = df['Label'].astype(int).to_numpy()

# Applying Anomaly Detector
output = run_Unsupervise_AD(AD_Name, data)

# Choosing anomaly score 
#score_type = "timestamp" #"timestamp", "feature", "time-feature"

#if score_type == "timestamp":
#    score = output.decision_function(data)
#elif score_type == "feature":
#    score = output.feature_importance().mean(axis=1)
#elif score_type == "time-feature":
#    score = output.time_feature().mean(axis=(1, 2))
#else:
#    raise ValueError("Invalid score_type. Choose 'timestamp', 'feature', or 'time-feature'.")

#Evaluation
evaluation_result = get_metrics(output, label)

INFO:p-3479:t-8613240576:get_model.py:get_model:Loading model from: ibm-granite/granite-timeseries-ttm-r2


[Zero] Reconstructing DataFrame
[Zero] Initializing preprocessor
[Zero] Loading model


/Users/ayumikatsuya/Documents/TSB-AD/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:p-3479:t-8613240576:get_model.py:get_model:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = main.
INFO:p-3479:t-8613240576:get_model.py:get_model:[TTM] context_length = 512, prediction_length = 96


[Zero] Creating datasets
[Zero] Training


/Users/ayumikatsuya/Documents/TSB-AD/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


++++++++++++++++++++ Test MSE zero-shot ++++++++++++++++++++


{'eval_loss': 0.24712394177913666, 'eval_runtime': 29.0809, 'eval_samples_per_second': 65.507, 'eval_steps_per_second': 65.507}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[Zero] Predicting
[Zero] Extracting targets
[Zero] Calculating mean squared error
[Zero] Padding complete


In [2]:
print(evaluation_result)

{'AUC-PR': 0.05865293264663233, 'AUC-ROC': 0.4332837565069585, 'VUS-PR': np.float64(0.08075563842024386), 'VUS-ROC': np.float64(0.4398316586694003), 'Standard-F1': np.float64(0.11080567919762975), 'PA-F1': 0.0, 'Event-based-F1': np.float64(0.0), 'R-based-F1': 0, 'Affiliation-F': 0.032788598390654976}


In [1]:
import pandas as pd
from TSB_AD.model_wrapper import run_Semisupervise_AD
from TSB_AD.evaluation.metrics import get_metrics

# Specify Anomaly Detector to use and data directory
AD_Name ='TTM_FT'
data_direc = 'Datasets/TSB-AD-M/115_PSM_id_1_Facility_tr_50000_1st_129872.csv'

# Load data
df = pd.read_csv(data_direc).iloc[120001:140000].dropna() 
data = df.iloc[:, 0:-1].values.astype(float)
label = df['Label'].astype(int).to_numpy()

# Split into few-shot (train) and test sets
fewshot_percent = 10
n_fewshot = int(len(data) * fewshot_percent / 100)
data_train = data[:n_fewshot]
label_train = label[:n_fewshot]
data_test = data[n_fewshot:]
label_test = label[n_fewshot:]

# Apply Semi-Supervised Anomaly Detector
output = run_Semisupervise_AD(
    AD_Name,
    data_train,
    data_test
)

# Evaluation on test set
evaluation_result = get_metrics(output, label_test)
print(evaluation_result)

INFO:p-2545:t-8613240576:get_model.py:get_model:Loading model from: ibm-granite/granite-timeseries-ttm-r2


[TTM] Reconstructing DataFrame
[TTM] Initializing preprocessor
[TTM] Loading model


/Users/ayumikatsuya/Documents/TSB-AD/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:p-2545:t-8613240576:get_model.py:get_model:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = main.
INFO:p-2545:t-8613240576:get_model.py:get_model:[TTM] context_length = 512, prediction_length = 96


[TTM] Creating datasets
[TTM] Using provided learning rate: 0.001
[TTM] Training


/Users/ayumikatsuya/Documents/TSB-AD/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
INFO:p-2645:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2647:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2648:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2651:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2653:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2654:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO

...

INFO:p-2698:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2699:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2701:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2702:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2703:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2704:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2705:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2706:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.


{'eval_loss': 0.5132123231887817, 'eval_runtime': 56.219, 'eval_samples_per_second': 1.868, 'eval_steps_per_second': 1.868, 'epoch': 1.0}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[TTM] Predicting


INFO:p-2720:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2721:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2722:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2724:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2726:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2727:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2728:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.
INFO:p-2729:t-8613240576:config.py:<module>:PyTorch version 2.3.0 available.


[TTM] Extracting targets
[TTM] Calculating mean squared error
[TTM] timestamp scores
[TTM] feature scores
[TTM] time-feature scores
[TTM] Padding complete


ValueError: Found input variables with inconsistent numbers of samples: [18000, 1999]

In [1]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

PyTorch version: 2.3.0
CUDA available: False
CUDA version: None
cuDNN version: None


In [2]:
import torch
try:
    print("CUDA available:", torch.cuda.is_available())
    print("Device count:", torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name(0))
except Exception as e:
    print("CUDA error:", e)

CUDA available: True
Device count: 1
Device name: NVIDIA RTX 2000 Ada Generation Laptop GPU
